## Лабораторная работа 1 модуль
### Ксения Анисимова ЦМГН 
#### Вариант 14

### Класс для габаритов и координат (в см)

In [391]:
from typing import Union

class Coords:
    def __init__(self, x: Union[int, float], y: Union[int, float], z: Union[int, float]):
        self.x = x
        self.y = y
        self.z = z
        
    def __str__(self):  
        return "(x-%s; y-%s; z-%s)" % (self.x, self.y, self.z)  #подумать над разделением

In [387]:
d = Coords(1, 2, 3)
print(d)

(x-1; y-2; z-3)


### Класс кухонного объета: 
**Координаты**: середина дна гарнитура  
**Габариты**: x - длина, y - высота, z - ширина  
**Методы проверки пересечения**

In [399]:
class Kitchen_Object:
    def __init__(self, size: Coords, coords: Coords):
        self.size = size
        self.coords = coords
    
    #метод нахождения координат двух углов проекции (дна) объекта (на одной из диагонали)
    @classmethod
    def corner_coords(self, size, coords):
        x = [coords.x - size.x/2, coords.z + size.z/2]
        z = [coords.x + size.x/2, coords.z - size.z/2]
        return x, z
    
    #координаты середины сторон гарнитура (без высоты) - для проверки, приставлен ли объект к стене 
#     @classmethod
#     def middle_coords(self, size, coords):
#         a = [coords.x - size.x/2, coords.z]
#         b = [coords.x, coords.z + size.z/2]
#         c = [coords.x + size.x/2, coords.z]
#         d = [coords.x, coords.z - size.z/2]
#         return [a, b, c, d]
    
    #проверка пересечения проекций 
    @classmethod
    def _bottom_overlap(cls, size1, size2, coords1, coords2):
        coords1 = Kitchen_Object.corner_coords(size1, coords1) # координаты по обеим концам диагонали
        top1, bottom1 = coords1[0], coords1[1] # координаты "верхней" и "нижней" точки проекции гарнитура(x;z)
        coords2 = Kitchen_Object.corner_coords(size2, coords2)
        top2, bottom2 = coords2[0], coords2[1]
        if top1[0] == bottom1[0] or top1[1] == bottom1[1] or top2[0] == bottom2[0] or top2[1] == bottom2[1]: #площадь 0
            return False
        if top1[0] > bottom2[0] or top2[0] >  bottom1[0]: #слева/справа
            return False
        if bottom1[1] > top2[1] or bottom2[1] > top1[1]: #сверху/снизу 
            return False
        return True
    
    #проверка пересечения по высоте
    def _height_overlap(self, size1, size2, coords1, coords2):
        height1 = size1.y #
        height2 = size2.y
        y1 = coords1.y
        y2 = coords2.y
        if y1 + height1 < y2:
            return False
        if y2 + height2 < y1:
            return False 
        return True
    
    @classmethod
    def overlap(self, object1, object2):
        if self._bottom_overlap(object1.size, object2.size, object1.coords, object2.coords) == False:
            return False
        if self._height_overlap(self, object1.size, object2.size, object1.coords, object2.coords):
            return True

### Класс шкафа: 
**Координаты и габариты** - наследованы от Kitchen_Object \
Материал сделан приватным из житейской логики

In [423]:
from typing import Union

class Cabinet(Kitchen_Object):
    def __init__(self, material, size, coords):
        super().__init__(size, coords)
        self.material = material

    def __repr__(self):  
        return "Cabinet: material - %s, size%s, coordinates%s)" % (self.material, self.size, self.coords)

    @property
    def material(self):
        return self.__material
    
    @material.setter 
    def material(self, material: Union[str, int]):
        self.__material = material

In [424]:
cab1 = Cabinet("wood", Coords(50,100,50), Coords(25,0,25))
cab2 = Cabinet("oak", Coords(30,50,30), Coords(25,10,100))
cab3 = Cabinet("birch", Coords(30,50,30), Coords(25,10,50))
print(cab1, cab2, cab3, sep="\n")
print(Kitchen_Object.overlap(cab1, cab2))
print(Kitchen_Object.overlap(cab1, cab3))

Cabinet: material - wood, size(x-50; y-100; z-50), coordinates(x-25; y-0; z-25))
Cabinet: material - oak, size(x-30; y-50; z-30), coordinates(x-25; y-10; z-100))
Cabinet: material - birch, size(x-30; y-50; z-30), coordinates(x-25; y-10; z-50))
False
True


### Класс техники: 
**Координаты и габариты** - наследованы от Kitchen_Object  
Имя сделано приватным из житейской логики  
Необходимость и смысл метода включения и выключения техники неясны, поэтому он не реализован

In [432]:
class Appliance(Kitchen_Object):
    def __init__(self, name, size, coords):
        super().__init__(size, coords)
        self.__name = name

    @property
    def name(self):
        return self.__name
    
    @name.setter 
    def name(self, name: Union[str, int]):
        self.__name = name

    def __repr__(self):  
        return  "Appliance: name - %s, size%s, coordinates%s)" % (self.name, self.size, self.coords)
    
    #не поняла, зачем тут метод включения/выключения техники
    def switch_on_off(self):
        pass

In [431]:
app1 = Appliance("TV", Coords(20,20,40), Coords(0,50,60))
app2 = Appliance("Oven", Coords(30,10,30), Coords(30,50,60))
print(app1)
print(app2)
print(Appliance.overlap(cab1, cab2))

Appliance: name - TV, size(x-20; y-20; z-40), coordinates(x-0; y-50; z-60))
Appliance: name - Oven, size(x-30; y-10; z-30), coordinates(x-30; y-50; z-60))
False


### Класс кухни: 
**Габариты (х, y)** - высоту не учитываем  
**Содержание** - список шкафов и техники

In [549]:
from collections import namedtuple
import math

kitchen_size = namedtuple('KitchenSize', ['x', 'y'])

class Kitchen:
    def __init__(self, size: kitchen_size, content: list):
        self.__size = size
        self.content = content 
        
    @property
    def size(self):
        return self.__size
    
    @size.setter 
    def size(self, size: kitchen_size):
        self.__size = size
     
    #def __find_appliance(self, content: list):
    def find_appliance(self, content: list):
        appliances = []
        for x in content:
            if isinstance(x, Appliance):
                appliances.append(x)
        return appliances
    
    #def __find_cabinet(self, content):
    def find_cabinet(self, content):
        cabinets = []
        for x in content:
            if isinstance(x, Cabinet):
                cabinets.append(x)
        return cabinets
    
# Метод для проверки, не висит ли техника в воздухе:
#     - если координата по высоте = 0, то не висит
#     - если больше нуля, то проверяем, не стоит ли техника на шкафе
#     - пока будем считать, что техника не может висеть на стене (но есть заготовка см. класс Kitchen_Object)

    def appliance_float(self):
        res = []
        appliance = self.find_appliance(self.content)
        cabinets = self.find_cabinet(self.content)
        for app in appliance:
            if app.coords.y == 0:
                res.append("%s doesn't float" % app.name)
            else:
                counter = 0
                for cab in cabinets:
                    if Kitchen_Object._bottom_overlap(cab.size, app.size, cab.coords, app.coords) and \
                    app.coords.y == cab.size.y + cab.coords.y or app.coords.x == 0 or app.coords.z == 0:
                        counter = counter + 1
                if counter >= 1:
                    res.append("%s doesn't float" % app.name)
                else:
                    res.append("%s floats!" % app.name)
        return res 
    
    def same_material_cabinets_too_close(self):
        res = set()
        cabinets = self.find_cabinet(self.content)
        for i in range(0, len(cabinets), 1):
            for j in range(0, len(cabinets), 1):
                if cabinets[i].material == cabinets[j].material and i != j \
                and math.dist([cabinets[i].coords.x, cabinets[i].coords.z], [cabinets[j].coords.x, cabinets[j].coords.z]) \
                < 200:
                    res.add(cabinets[i].material)
        for x in res:
            print("Cabinets of %s are too close!" % x)
            
    def overlap(self):
        counter = 0 
        res = set()
        for x in self.content:
            for y in self.content:
                if x != y and Kitchen_Object.overlap(x, y) == True:
                    if (y, x) not in res:
                        res.add((x, y))
        return res

In [550]:
def print_set(res):
    for x in res:
        print(*x, sep="\n")

### Создаем тестовую мебель и технику для кухни

In [562]:
cabinet1 = Cabinet("oak", Coords(1, 3, 2), Coords(0.5, 0, 3))
cabinet2 = Cabinet("oak", Coords(2, 3, 1), Coords(2, 0, 0.5))
appl1 = Appliance("TV", Coords(1, 1, 1), Coords(2.5, 3, 0.5))
appl2 = Appliance("Oven", Coords(1, 1, 1), Coords(4.5, 3, 2.5))
appl3 = Appliance("Oven2", Coords(1, 1, 1), Coords(3, 0, 4.5))

### Создаем тестовую кухню и проверяем по условиям задачи

In [565]:
content = [cabinet1, cabinet2, appl1, appl2, appl3]
size = kitchen_size(100, 100)
k = Kitchen(size, content)

print("\033[1m" + "\nAppliances position:" + '\033[0m')
print(*k.appliance_float(), sep="\n")

print("\033[1m" + "\nCabinets of the same material that are less than 2m close:" + '\033[0m')
k.same_material_cabinets_too_close()

print("\033[1m" + "\nPieces of furniture that overlap:" + '\033[0m')
print_set(k.overlap())


Appliances position:
TV doesn't float
Oven floats!
Oven2 doesn't float

Cabinets of the same material that are less than 2m close:
Cabinets of oak are too close!

Pieces of furniture that overlap:
Cabinet: material - oak, size(x-2; y-3; z-1), coordinates(x-2; y-0; z-0.5))
Appliance: name - TV, size(x-1; y-1; z-1), coordinates(x-2.5; y-3; z-0.5))
